## 激活函数  
___
### 神经网络中的激活函数需要满足的条件：
1. 非线性：即导数不是常数。这个条件可以保证多层网络不退化成单层线性网络。 


2. 几乎处处可微：可微保证了在优化中梯度的课计算性。


3. 计算简单：激活函数在神经网络前向的计算次数与神经元的个数成正比，因此简单的非线性函数自然更适合用作激活函数。


4. 非饱和性：饱和性是指在某些区间梯度接近于零（即梯度消失），使得参数无法继续更新。
    * sigmod函数：它的导数在x为比较大的正值和比较小的负值时都会接近于0。
    * 阶跃函数：几乎所有位置梯度都为0，所以无法作为激活函数。
    * ReLU函数：在x>0时梯度恒为1，不会饱和，但在x<0时，梯度恒为0，出现饱和。


5. 单调性：即导数符号不变


6. 输出范围有限：有限的输出范围使得网络对于一些较大的输入也会比较稳定。


7. 接近恒等变换：即约等于x。这样的好处是使得输出的幅值不会随着深度的增加而发生显著的增加，从而使网络更为稳定。这个与非线性是有点矛盾的，因此激活函数基本只是部分满足这个条件，比如TanH只在原点附近有线性区（在原点为0且在原点的导数为1），而ReLU只在x>0时为线性。这个性质也让初始化参数范围的推导更为简单。


8. 参数少


9. 归一化：这个是最近才出来的概念，对应的激活函数是SELU，主要思想是使样本分布自动归一化到零均值、单位方差的分布，从而稳定训练。在这之前，这种归一化的思想也被用于网络结构的设计，比如Batch Normalization。


10. zero-centered：Sigmoid函数的输出值恒大于0，这会导致模型训练的收敛速度变慢。

___
### 常用的激活函数：
**1. sigmod函数**  


\begin{equation}
g(z)=\frac{1}{1+e^{-z}}
\end{equation}


\begin{equation}
g'(z)=g(z)[1-g(z)]
\end{equation}


**2. tanh函数**


\begin{equation}
g(z)=\frac{e^z-e^{-z}}{e^z+e^{-z}}
\end{equation}


\begin{equation}
g'(z)=1-[g(z)]^2
\end{equation}


**3. ReLU函数** 


\begin{equation}
g(z)=max(0,z)
\end{equation}


\begin{equation}
g'(z)=
\begin{cases}
0,\quad if \quad z<0\\
1,\quad if \quad z\ge0
\end{cases}
\end{equation}


**4. Leaky ReLU函数**


\begin{equation}
g(z)=max(0.01z,z)
\end{equation}


\begin{equation}
g'(z)=
\begin{cases}
0.01,\quad if \quad z<0\\
1,\quad if \quad z\geq0
\end{cases}
\end{equation}
___



## 神经网络的训练
### 前向传播：
\begin{equation}
Z^{[1]}=w^{[1]}x+b^{[1]}
\end{equation}
\begin{equation}
A^{[1]}=g^{[1]}(Z^{[1]})\\
Z^{[2]}=w^{[2]}A^{[1]}+b^{[2]}\\
A^{[2]}=g^{[2]}(Z^{[2]})
\end{equation}
___
### 反向传播：
1. Lose Function:$$L=-\frac 1m \sum _{i=0}^{m-1}[(1-y_i)log(1-\hat y_i)+y_ilog(\hat y_i)]$$
2. 计算$dw^{[2]},db^{[2]}:$
\begin{equation}
dZ^{[2]}=\frac 1m (A^{[2]}-Y),\qquad Y=[y^{(1)},y^{(2)},...,y^{(m)}]\\
dw^{[2]}=\frac 1m dZ^{[2]}A^{[1]T}\\
db^{[2]}=\frac 1m dZ^{[2]}
\end{equation}
3. 计算$dw^{[1]},db^{[1]}:$
\begin{equation}
dZ^{[1]}=W^{[2]T}dZ^{[2]}*A^{[1]'}(Z^{[1]})\\
dw^{[1]}=dZ^{[1]}x^T\\
db^{[1]}=dZ^{[1]}.sum(axix=1)
\end{equation}
___
### 维数计算：
输入 x：（data_dim, data_num）  
隐藏层 w1_array：（n1, data_dim）  
b1：（n1, 1）  
z1, a1：（n1, data_num）
w2_array：（n2, n1）  
b2：（n2, 1）  
z2, a2：（n2, data_num）

In [49]:
# -*- coding: utf-8 -*- 
import numpy as np
from math import *

class BPANN(object):
    def __init__(self, layer, *layer_nodes):
        self.layer = layer
        
        if layer != len(layer_nodes):
            print("Can't initial the neuron network!")
            exit(1)
        
        self.layer_nodes = layer_nodes
        
    def load_data(self, data_set, labels):
        self.data_set = np.mat(data_set).T
        self.labels = np.mat(labels)
        self.data_size = len(data_set)
        self.data_dim = len(data_set[0])
        self.w1_array = np.mat(np.random.randn(self.layer_nodes[0], self.data_dim) * 0.01)
        self.w2_array = np.mat(np.random.randn(self.layer_nodes[1], self.layer_nodes[0]) * 0.01)
        self.b1 = np.mat(np.zeros(self.layer_nodes[0])).T
        self.b2 = np.zeros(self.layer_nodes[1])
        
    def __for_prop(self, x):
        # forward propagation
        z1 = np.dot(self.w1_array, np.mat(x).T) + self.b1
        # print(self.w1_array, self.b1, z1)
        a1 = 1 / (1 + np.exp(-z1))
        z2 = np.dot(self.w2_array, a1) + self.b2
        a2 = 1 / (1 + np.exp(-z2))
        return a2
    
    def training(self, iterate = 100):
        for i in range(iterate):
            z1 = self.w1_array * self.data_set + self.b1
            a1 = 1 / (1 + np.exp(-z1))
            z2 = self.w2_array * a1 + self.b2
            a2 = 1 / (1 + np.exp(-z2))
            
            dz2 = a2 - self.labels
            db2 = 1 / self.data_size * dz2.sum()
            dw2 = 1 / self.data_size * dz2 * a1.T
            dz1 = 1 / self.data_size * np.multiply(self.w2_array.T * dz2, np.multiply(a1, (1 - a1)))
            dw1 = dz1 * self.data_set.T
            db1 = dz1.sum(axis = 1)
            
            self.w2_array -= dw2
            self.b2 -= db2
            self.w1_array -= dw1
            self.b1 -= db1
        
        #print(self.w1_array, self.b1, self.w2_array, self.b2)
        
    def predict(self, x):
        if self.__for_prop(x) >= 0.5:
            return 1
        else:
            return 0
        
def main():
    data_set = [[3, 2],
               [1, 1],
               [2, 1]]
    labels = [1, 0, 1]
    bpan = BPANN(2, 4, 1)
    bpan.load_data(data_set, labels)
    bpan.training()
    print(bpan.predict([1,1]))
    
if __name__ == "__main__":
    main()

0


In [42]:
a = np.mat([[1,2,3],
           [2,3,4]])
c = np.mat([[1],[1]])
print()
# print(np.exp(a))